## Uso de Aprendizado Profundo na classificação de imagens de histopatológicas

### Contextualização

   <p>Segundo o INCA(Instituto Nacional de Câncer) (2020), o <b>Câncer de Mama</b> é uma doença causada pela multiplicação desordenada de células da mama. Esse processo gera células anormais que se multiplicam, formando um tumor. Neste contexto, há vários tipos de câncer de mama. Ocasionando, uma possibilidade de evolução de diferentes formas.</p>
    <p>Assim, alguns tipos podem apresentar um rápido desenvolvimento, enquanto outros podem crescem de forma mais lenta. A diferença entre tais comportamentos se deve a características próprias de cada tumor.Além disso, este tipo de  câncer também acomete homens, sendo de ocorrência rara, representando apenas 1% do total de casos da doença.</p>
    <p> Ademais, a <b>Histopatologia</b> ou <b>histologia patológica</b> é o estudo de como uma doença específica afeta um conjunto de células. Seu nome é formado da combinação entre três palavras gregas: 'histo', 'pathos' e 'logia'. Geralmente, é feito um estudo da biópsia usando um microscópio e corantes.
    Em análise, esse trabalho irá realizar classificação binária de imagens histopatopatológicas, utilizando Redes Neurais Convolucionais e técnicas de Aprendizado profundo. 
    </p>


### Conjunto de dados - BreaKHis

<p>O conjunto de dados é oriundo da plataforma Kaggle. Onde, contém imagens de biópsia microscópica de tumores benignos e malignos da mama. Neste conjunto de dados foram feitas algumas modificações, como: separar dados de treinamento e dados de teste com pastas diferentes, cada arquivo representa partes diferentes de cada amostra. Além disso, há apenas uma amostra parcial do zoom óptico de 400x, para mais interesse no conjunto de dados, consulte este artigo: </p>

<p>FA Spanhol, LS Oliveira, C. Petitjean and L. Heutte, "A Dataset for Breast Cancer Histopathological Image Classification," in IEEE Transactions on Biomedical Engineering, vol. 63, no. 7, pp. 1455-1462, July 2016, doi: 10.1109 / TBME.2015.2496264. </p>


<img src='BreaKHis 400X/test/malignant/SOB_M_DC-14-10926-400-004.png' width="300px" align="center"> 
<center>Fonte: Dataset BreaKHis.</center>


### Codificação 

#### Importação iniciais

In [38]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import  BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd

O Keras é uma biblioteca de rede neural de código aberto escrita em Python, que roda em cima de TensorFlow. Dentre as importações:
* **Sequencial** - fornece recursos de treinamento e inferência neste modelo;
* **Conv2D** - cria um kernel de convolução que é convolvido com a entrada da camada para produzir um tensor de saídas;
* **MaxPooling2D** - é um tipo de operação que normalmente é adicionado a CNNs seguindo camadas convolucionais individuais;
* **Flatten** - remodela os dados de entrada em um formato adequado para as camadas convolucionais;
* **Dense** - calcula uma função de ativação em conjunto com os dados de entrada e pesos;
* **Dropout** -  define aleatoriamente as unidades de entrada para 0 com uma frequência de taxa em cada etapa durante o tempo de treinamento, o que ajuda a evitar overfitting;
* **BatchNormalization** - aplica uma transformação que mantém a saída média próxima a 0 e o desvio padrão da saída próximo a 1;
* **ImageGenerator** - transforma os dados originais aleatoriamente e retorna apenas os dados novos transformados.

#### Definição da Rede Neural

In [2]:
classificador = Sequential()
classificador.add(Conv2D(64, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size=(2,2)))


classificador.add(Conv2D(64, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size=(2,2)))

classificador.add(Flatten())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Na definição da Rede Neural, temos:
* criação de uma rede sequencial;
* camada de convolução, com filtros, kernels, formato das entradas e função de ativação;
* camada de BatchNormalization;
* camada de MaxPooling2D, com definição do tamanho do pool(janela);
* camada flatten.

#### Criação da Rede Neural densa

In [3]:
classificador.add(Dense(units=128, activation='relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 1, activation = 'sigmoid'))

classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Na criação da Rede Neural Densa:
* camadas densas, com a quantidade de unidades e a função de ativação;
* camadas de Dropout;
* camada de densa de saída, com função de ativação;
* compilação das camadas, juntamente com seus argumentos.

In [4]:
gerador_treinamento = ImageDataGenerator(rescale=1./255,
                                           rotation_range=7,
                                           height_shift_range=0.07,
                                           shear_range=0.2,
                                           zoom_range=0.2,
                                           horizontal_flip=True)


No ImageDataGenerator:
* rescale - fator de reescalonamento;
* rotation_range - faixa de graus para rotações aleatórias;
* horizontal_flip - inverte aleatoriamente as entradas horizontalmente;
* shear_range - intensidade de cisalhamento (ângulo de cisalhamento no sentido anti-horário em graus);
* height_shift_range - faixa de mudança de altura;
* zoom_range - intervalo para zoom aleatório.

In [5]:
gerador_teste = ImageDataGenerator(rescale = 1./255)


#### Leitura da base de dados de treinamento

In [9]:
base_treinamento = gerador_treinamento.flow_from_directory('BreaKHis 400X/train',
                                                           target_size = (64,64),
                                                           batch_size = 16,
                                                           class_mode = 'binary')

Found 1148 images belonging to 2 classes.


#### Leitura da base de dados de teste

In [10]:
base_test = gerador_teste.flow_from_directory('BreaKHis 400X/test',
                                              target_size = (64,64),
                                              batch_size = 16,
                                              class_mode = 'binary')

Found 545 images belonging to 2 classes.


#### Treinamento da rede

Treinamento da rede, com:

* **steps_per_epoch** - número total de etapas (lotes de amostras) para gerar do gerador antes de declarar que uma época terminou e iniciar a próxima época;
* **epochs** - iterações para ajuste dos pesos;
* **validation_data** - conjunto de validação dos dados;
* **validation_steps** - especifica o número total de etapas tomadas a partir do gerador antes de ser interrompido em cada época e seu valor é calculado como o número total de pontos de dados de validação;

In [13]:
classificador.fit_generator(base_treinamento, steps_per_epoch = 1693,
                            epochs = 5, validation_data = base_test,
                            validation_steps = 1693)

Epoch 1/5
1693/1693 [==============================] - 1112s 657ms/step - loss: 0.1913 - acc: 0.9246 - val_loss: 0.9677 - val_acc: 0.8277
Epoch 2/5
1693/1693 [==============================] - 1004s 593ms/step - loss: 0.1363 - acc: 0.9461 - val_loss: 0.6397 - val_acc: 0.8018
Epoch 3/5
1693/1693 [==============================] - 797s 471ms/step - loss: 0.1184 - acc: 0.9545 - val_loss: 0.5931 - val_acc: 0.7999
Epoch 4/5
1693/1693 [==============================] - 796s 470ms/step - loss: 0.0935 - acc: 0.9660 - val_loss: 1.2702 - val_acc: 0.8020
Epoch 5/5
1693/1693 [==============================] - 796s 470ms/step - loss: 0.0795 - acc: 0.9724 - val_loss: 1.6310 - val_acc: 0.7781


In [43]:
steps = np.math.ceil(base_test.samples / base_test.batch_size)

In [44]:
predicoes = classificador.predict_generator(base_test, steps=steps)

In [45]:
predicao_classes = np.argmax(predicoes, axis=1)

In [46]:
X_classes = base_test.classes


In [47]:
classes_rotuladas = list(base_test.class_indices.keys())   

In [48]:
report = metrics.classification_report(X_classes, predicao_classes, target_names=classes_rotuladas)
print(report)    

              precision    recall  f1-score   support

      benign       0.32      1.00      0.49       176
   malignant       0.00      0.00      0.00       369

   micro avg       0.32      0.32      0.32       545
   macro avg       0.16      0.50      0.24       545
weighted avg       0.10      0.32      0.16       545



/home/ana/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Fontes
https://www.inca.gov.br/tipos-de-cancer/cancer-de-mama <br>
https://www.kaggle.com/forderation/breakhis-400x <br>
https://pt.wikipedia.org/wiki/Histopatologia